In [42]:
import pickle
#import cPickle as pickle
import os.path
import numpy as np
from gensim.models import word2vec
from seq2seq.models import SimpleSeq2Seq
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

def tmpdir(path):
    return "{0}/tmp2/{1}".format(os.environ['HOME'], path)

# Constants and Helpers
MODEL_NAME = "chat_bot_english"
SEQ2SEQ_MODEL_PATH = "seq2seq2_model"
DATA_FILE_PATH = "../../dont_remove_data/twitter_en_tmp_16000.txt"
#DATA_FILE_PATH = "../../dont_remove_data/twitter_en.txt"
WORD_VEC_CACHE_PATH = tmpdir("chat_bot_word_vec_model.saved")
SANITIZED_TEXT_PATH = tmpdir("sanitized_text.saved")
QUESTION_LINES_VAL_PATH = tmpdir("question_lines_val.saved")
ANSWER_LINES_VAL_PATH = tmpdir("answer_lines_val.saved")
QUESTION_LINES_TRAIN_PATH = tmpdir("question_lines_train.saved")
ANSWER_LINES_TRAIN_PATH = tmpdir("answer_lines_train.saved")

SANITIZED_VEC_MODEL_CACHE_PATH = tmpdir("chat_bot_sanitized_word_vec_model.saved")
EN_WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyz ' # space is included in whitelist

# Constant tokens
UNK = 'unk'
EOS = 'EOS'

# Model parameters
WORD_VEC_SIZE = 1024
SEQ_LEN = 20

def read_lines(filename):
    return open(filename).read().split('\n')[:-1]

def filter_line(line, whitelist):
    return ''.join([ ch for ch in line if ch in whitelist ])

# return variable name of obj
def namestr(obj):
    return [name for name in vars() if vars()[name] is obj][0]

def save_obj(key, obj):
    print(">saving {0} to file".format(tmpdir(key)))
    with open(tmpdir("{0}.saved".format(key)), mode='wb') as f:
        pickle.dump(obj, f, protocol=0)

def load_obj(key, obj):
    filepath = tmpdir("{0}.saved".format(key))
    if os.path.exists(filepath):
        print(">loading {0} from file".format(key))
        with open(filepath, mode='rb') as f:
            return pickle.load(f)
    else:
        print(">{0} not found.".format(key))        
        return False
    
def read_filtered_lines(path):
    print(">reading file...")    
    lines = read_lines(path)
    print(">removing unnecesary characters")    
    return [filter_line(line, EN_WHITELIST) for line in lines]

def build_word_vec(path, word_vec_size=WORD_VEC_SIZE, min_count=20, window=15):
    sentences = word2vec.LineSentence(path)
    word_vec = word2vec.Word2Vec(sentences, size=word_vec_size, min_count=min_count, window=window)    
    return word_vec

def load_or_build_word_vec(vocab_path, cache_path, enable_cache):
    if enable_cache and os.path.exists(cache_path):
        print(">loading from exisiting model")
        return word2vec.Word2Vec.load(cache_path)
    else:
        print(">creating word_vec model")
        word_vec = build_word_vec(vocab_path)
        word_vec.save(cache_path)
        return word_vec
    
def sanitize_lines(lines, word_vec):
    sanitized_lines = []    
    for line in lines:
        sanitized_line = []
        for word in line.split():
            if word in word_vec.wv:
                sanitized_line.append(word)
            else:
                sanitized_line.append(UNK)
        sanitized_line.append(EOS)
        sanitized_lines.append(' '.join(sanitized_line))
    return sanitized_lines

def load_or_build_sanitized_lines(lines, word_vec, enable_cache):
    sanitized_lines = False
    key = "sanitized_lines"
    if enable_cache:
        sanitized_lines = load_obj(key, sanitized_lines)
    if sanitized_lines:
        return sanitized_lines
    ret = sanitize_lines(lines, word_vec)
    save_obj(key, ret)
    return ret    

def save_lines(path, lines):
    f = open(path, 'w')
    for line in lines:
        f.write(line)
        f.write('\n')
    f.close()    

def split_lines(lines):
    question_lines = []
    answer_lines = []
    print(">splinting data...")
    # 2n   lines are questions
    # 2n+1 lines are answers
    for i in range(0, len(lines) // 2, 2):
        question_lines.append(lines[i])
        answer_lines.append(lines[i + 1])
    return (question_lines, answer_lines)    

def shape_data(lines, nb_lines, word_vec):
    zero_vec = np.zeros((WORD_VEC_SIZE))
    ret = []
    for line in lines[:nb_lines]:
        sentence = []
        words = line.split()
        for i in range(0, SEQ_LEN):
            if i < len(words):
                sentence.append(word_vec.wv[words[i]])
            else:
                sentence.append(zero_vec)            
        ret.append(sentence)
    return np.array(ret)

def build_train_data(question_lines, answer_lines, nb_train, word_vec):
    return (shape_data(question_lines, nb_train, word_vec), 
            shape_data(answer_lines, nb_train, word_vec))

def weights_dir(model_version):
    return "{0}/Dropbox/{1}-{2}".format(os.environ['HOME'], MODEL_NAME, model_version)

def best_weights_path(model_version):
    files = os.listdir(weights_dir(model_version))
    files.sort()
    return "{0}/{1}".format(weights_dir(model_version), files[0])

def default_callback_list(model_version):
    os.makedirs(weights_dir(model_version), exist_ok=True)
    filepath = weights_dir(model_version) + "/{loss:.4f}"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    return [checkpoint]


In [43]:
# Create Word Vector for all vocaburaries in the whole data
print(">buidling word vec")
word_vec = load_or_build_word_vec(DATA_FILE_PATH, WORD_VEC_CACHE_PATH, enable_cache=True)
print("word_vec", word_vec)

>buidling word vec
>creating word_vec model
word_vec Word2Vec(vocab=1049, size=1024, alpha=0.025)


In [44]:
data_lines = read_filtered_lines(DATA_FILE_PATH)
print("data_lines[10]", data_lines[10])


>reading file...
>removing unnecesary characters
data_lines[10] jill stein amp her fan club can now officially go to hell just vote trump amp be done with it


In [45]:
print(">sanitizing lines")
sanitized_lines = load_or_build_sanitized_lines(data_lines, word_vec, enable_cache=True)
question_lines, answer_lines = split_lines(sanitized_lines)
print(len(sanitized_lines))
print("question_lines[15]=", question_lines[15])
print("answer_lines[15]=", answer_lines[15])


# 10%
nb_validation = int(len(question_lines) * 0.1)

question_lines_train = question_lines[nb_validation:]
question_lines_val = question_lines[:nb_validation]

answer_lines_train = answer_lines[nb_validation:]
answer_lines_val = answer_lines[:nb_validation]

save_lines(QUESTION_LINES_VAL_PATH, question_lines_val)
save_lines(ANSWER_LINES_VAL_PATH, answer_lines_val)
save_lines(QUESTION_LINES_TRAIN_PATH, question_lines_train)
save_lines(ANSWER_LINES_TRAIN_PATH, answer_lines_train)


# Save the sanitized text to file, so that we can use it for WordVec
save_lines(SANITIZED_TEXT_PATH, sanitized_lines)
print(SANITIZED_TEXT_PATH, "wrritten as file")

sanitized_lines = None
data_lines = None
word_vec = None

>sanitizing lines
>sanitized_lines not found.
>saving /Users/higepon/tmp2/sanitized_lines to file
>splinting data...
16000
question_lines[15]= wow this is such a unk photo what did you use to take it EOS
answer_lines[15]= on unk of unk thank you he unk a unk unk unk EOS
/Users/higepon/tmp2/sanitized_text.saved wrritten as file


In [46]:
# Create word_vec again, so that UNK token can also have vector representation
print(">buidling word vec with unk")
word_vec_with_unk = load_or_build_word_vec(SANITIZED_TEXT_PATH, SANITIZED_VEC_MODEL_CACHE_PATH, enable_cache=True)
print("word_vec_with_unk", word_vec_with_unk)

>buidling word vec with unk
>creating word_vec model
word_vec_with_unk Word2Vec(vocab=886, size=1024, alpha=0.025)


In [47]:
def shape_line(line, word_vec):
    zero_vec = np.zeros((WORD_VEC_SIZE))
    ret = []
    sentence = []
    words = line.split()
    for i in range(0, SEQ_LEN):
        if i < len(words):
            sentence.append(word_vec.wv[words[i]])
        else:
            sentence.append(zero_vec)            
    ret.append(sentence)
    return np.array(ret)

def generate_train_from_file(questions_path, answers_path, word_vec):
    while 1:
        with open(questions_path) as qf, open(answers_path) as af: 
            for q, a in zip(qf, af):
                x = shape_line(q, word_vec)
                y = shape_line(a, word_vec)
                yield (x, y)

In [48]:
print(">initiating model")
seq2seq = SimpleSeq2Seq(
    input_dim=WORD_VEC_SIZE, # same as the chatbot example
    input_length=SEQ_LEN, # same as the chatbot example
    hidden_dim=100, # todo
    output_dim=WORD_VEC_SIZE, # same as the chatbot example
    output_length=SEQ_LEN, # same as the chatbot example
    unroll=True,
    depth=3)
print(seq2seq)

print(">compiling seq2seq...")
#seq2seq.compile(loss='categorical_crossentropy', optimizer='adam')

seq2seq.compile(loss='mse', # same as the chatbot example
                optimizer='adam') # same as the chatbot example
print(">done")

>initiating model
>compiling seq2seq...
>done


In [49]:
# Train!
model_version = "0415-refined-param_32000"
train_generator = generate_train_from_file(QUESTION_LINES_TRAIN_PATH, ANSWER_LINES_TRAIN_PATH, word_vec_with_unk)
val_generator = generate_train_from_file(QUESTION_LINES_VAL_PATH, ANSWER_LINES_VAL_PATH, word_vec_with_unk)

samples_per_epoch = 200
nb_val_samples = int(samples_per_epoch * 0.1)

seq2seq.fit_generator(train_generator,
                      samples_per_epoch=samples_per_epoch,
                      nb_val_samples=nb_val_samples,
                      nb_epoch=100,
#                      verbose=1,
                      validation_data=val_generator,
                      callbacks=default_callback_list(model_version))



Epoch 1/100
200/200 [==============================] - 86s - loss: 0.0022 - val_loss: 0.0022
Epoch 2/100
200/200 [==============================] - 83s - loss: 0.0018 - val_loss: 0.0017
Epoch 3/100
200/200 [==============================] - 86s - loss: 0.0019 - val_loss: 0.0018
Epoch 4/100
200/200 [==============================] - 78s - loss: 0.0017 - val_loss: 0.0012
Epoch 5/100
200/200 [==============================] - 81s - loss: 0.0018 - val_loss: 0.0016
Epoch 6/100
200/200 [==============================] - 74s - loss: 0.0018 - val_loss: 0.0020
Epoch 7/100
200/200 [==============================] - 76s - loss: 0.0018 - val_loss: 0.0015
Epoch 8/100
200/200 [==============================] - 79s - loss: 0.0019 - val_loss: 0.0015
Epoch 9/100
200/200 [==============================] - 80s - loss: 0.0019 - val_loss: 0.0015
Epoch 10/100
200/200 [==============================] - 76s - loss: 0.0018 - val_loss: 0.0017
Epoch 11/100
200/200 [==============================] - 76s - loss: 0

In [ ]:
seq2seq.save(SEQ2SEQ_MODEL_PATH)

In [28]:
import sys
sys.setrecursionlimit(100000)
seq2seq = load_model(SEQ2SEQ_MODEL_PATH)

Exception: Invalid layer: RecurrentContainer

In [13]:
# you want to turn twitter followers into blog readers
tt = seq2seq.predict(X_train[20].reshape(1, 20, word_vec_size))[0]
print(tt.shape)
for t in tt:
    print(word_vec_with_unk.similar_by_vector(t, topn=1)[0])


NameError: name 'X_train' is not defined

In [16]:
model_version = "0414-refined-param"
print("loading", best_weights_path(model_version))
seq2seq.load_weights(best_weights_path(model_version))  
print("loaded")

loading /Users/higepon/Dropbox/chat_bot_english-0414-refined-param/0.1338
loaded


In [52]:
texts = [
    'When is good for you today?',
    "I'm leaving soon.",
    "I'm tired"
]
filtered_texts = [filter_line(line.lower(), EN_WHITELIST) for line in texts]

print(filtered_texts)



['when is good for you today', 'im leaving soon', 'im tired']


In [54]:
def predictReply(text):
    text = filter_line(text.lower(), EN_WHITELIST)
    lines_with_unk = []
    for word in text.split():
        if word in word_vec_with_unk.wv:
            lines_with_unk.append(word)
        else:
            lines_with_unk.append(UNK)
    lines_with_unk.append(EOS)
    text_with_unk = " ".join(lines_with_unk)
    X = shape_data([text_with_unk], 1, word_vec_with_unk)
    Y = seq2seq.predict(X.reshape(-1, 20, WORD_VEC_SIZE))
    print(X.shape)
    predicted_words = []
    for word_vec in Y[0]:
        p = word_vec_with_unk.similar_by_vector(word_vec, topn=1)[0][0]
        if p == EOS:
            break
        else:
            predicted_words.append(p)
    return ' '.join(predicted_words)

source = "Hi there, how are you Taro?"
y = predictReply(source)
print(y)
source = source + y.split()[0]
print(source)
print(predictReply(source))
source = source + y.split()[0]
print(source)
print(predictReply(source))
source = source + y.split()[0]
print(source)
print(predictReply(source))
source = source + y.split()[0]
print(source)
print(predictReply(source))
source = source + y.split()[0]
print(source)
print(predictReply(source))
        

(1, 20, 1024)
better friends any those those other and and and and and and and and and and and and and and
Hi there, how are you Taro?better
(1, 20, 1024)
better friends any those those other and and and and and and and and and and and and and and
Hi there, how are you Taro?betterbetter
(1, 20, 1024)
better friends any those those other and and and and and and and and and and and and and and
Hi there, how are you Taro?betterbetterbetter
(1, 20, 1024)
better friends any those those other and and and and and and and and and and and and and and
Hi there, how are you Taro?betterbetterbetterbetter
(1, 20, 1024)
better friends any those those other and and and and and and and and and and and and and and
Hi there, how are you Taro?betterbetterbetterbetterbetter
(1, 20, 1024)
better friends any those those other and and and and and and and and and and and and and and


In [55]:
texts = []
for line in filtered_texts:
    sanitized_line = []
    for word in line.split():
        if word in word_vec_with_unk.wv:
            sanitized_line.append(word)
        else:
            sanitized_line.append(UNK)
    texts.append(' '.join(sanitized_line))
print("texts=", texts)
X_test = shape_data(texts, len(texts), word_vec_with_unk)
print("shaped\n")
Y_test = seq2seq.predict(X_test.reshape(-1, 20, WORD_VEC_SIZE))
for i, y in enumerate(Y_test):
    for word in y:
        print(word_vec_with_unk.similar_by_vector(word, topn=1)[0])

texts= ['when is good for you today', 'im leaving soon', 'im tired']
shaped

('better', 0.9955774545669556)
('friends', 0.9946251511573792)
('any', 0.9929155707359314)
('those', 0.99363112449646)
('those', 0.9928908944129944)
('other', 0.992620050907135)
('and', 0.9927718639373779)
('and', 0.9927970767021179)
('and', 0.993635356426239)
('and', 0.9933624267578125)
('and', 0.9935669898986816)
('and', 0.9932768940925598)
('and', 0.9928967952728271)
('and', 0.9926059246063232)
('and', 0.9923471212387085)
('and', 0.9918537139892578)
('and', 0.9908884763717651)
('and', 0.9893754720687866)
('and', 0.9873281717300415)
('and', 0.9850616455078125)
('better', 0.9955806732177734)
('friends', 0.9946293234825134)
('any', 0.992917537689209)
('those', 0.9936306476593018)
('those', 0.9928945302963257)
('other', 0.9926038384437561)
('and', 0.9927721619606018)
('and', 0.9927950501441956)
('and', 0.9936323165893555)
('and', 0.9933602809906006)
('and', 0.9935623407363892)
('and', 0.9932677149772644)
('and'

In [30]:
## EOS
## refactoring

## batch or generator使う
